# Introdução ao BERT

Neste notebook você verá um exemplo de como usar o [BERT](https://arxiv.org/abs/1810.04805) para extrair os embeddings de sentenças, além de conhecer mais sobre este modelo. 

Este tutorial foi adapato de:  BramVanroy / bert-for-inference (https://github.com/BramVanroy/bert-for-inference).

In [3]:
import torch
from transformers import BertModel, BertTokenizer

ImportError: DLL load failed while importing _C: The specified procedure could not be found.

## O tokenizador (tokenizer)

Os modelos deep learning trabalham com tensors, que são basicamente vetores, que por sua vez são um grupo de números. Para começar, o texto de entrada (string) precisa ser convertido em um tipo de data que os modelos possam usar (números). Essa é a tarefa do tokenizador. 

In [2]:
# Inicializando o tokenizador com um modelo pré-treinado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Durante o pré-treinamento, o tokenizador também é treinado, gerando um vocabulário conhecido. Cada palavra do vocabulário é atribuída a um índice (número), que pode ser usado pelo modelo. 

Para lidar com problemas das palavras que o tokenizador não conhece (out-of-vocabulary ou OOV), uma técnica é utilizada para garantir que o tokenizador aprenda "subpalavras". Desta forma, quando usamos os modelos pré-treinados, não teremos problemas de OOV. Quando o tokenizador não reconhece uma palavra, que não está no vocabulário, ele divide a palavra em pequenas unidades que são conhecidas. O tokenizador do BERT usa [WordPiece](https://arxiv.org/pdf/1609.08144.pdf) para dividir os tokens.

In [3]:
# Converte a string "granola bars" para um vocabulário tokenizado 
granola_ids = tokenizer.encode('granola bars')
# Print the IDs
print('granola_ids', granola_ids)
print('type of granola_ids', type(granola_ids))
# Converte os IDs to the actual vocabulary item
# Notice how the subword unit (suffix) starts with "##" to indicate 
# that it is part of the previous string
print('granola_tokens', tokenizer.convert_ids_to_tokens(granola_ids))

Você deve ter notado os tokens especiais [CLS] e [SEP]. Esses tokens são adicionados automaticamente pelo método `.encode()`, então não precisamos nos preocupar com eles. O primeiro é um token de classificação que foi pré-treinado, utilizado nas tarefas de classificação. Desta forma, ao invés de fazer a média de todos os tokens e usá-los como uma representação de frase, é recomendado apenas pegar a saída do [CLS] que representa a frase inteira. [SEP], por sua vez, é inserido como um separador entre várias instâncias, usado por exemplo na predição da próxima sentença, separando uma frase da outra.

Como vimos acima, o tipo de dados dos IDs de cada token é uma lista de inteiros. Neste notebook vamos usar a biblioteca `transformers` em combinação com PyTorch, que trabalha com tensores. Um tensor é um tipo especial de lista otimizada, normalmente usado em deep learning. Para converter nossos IDs dos tokens em um tensor, podemos simplesmente chamar um construtor de tensor passando a lista. Aqui, vamos usar um `LongTensor` que é usado para inteiros (para números de ponto flutuante,  usaríamos um `FloatTensor` ou apenas` Tensor`). 

O método `.encode ()` do tokenizer pode retornar um tensor em vez de uma lista, passando o parâmetro `return_tensors = 'pt'`, mas para fins de ilustração, faremos a conversão de uma lista para um tensor manualmente.

In [4]:
# Convert the list of IDs to a tensor of IDs 
granola_ids = torch.LongTensor(granola_ids)
# Print the IDs
print('granola_ids', granola_ids)
print('type of granola_ids', type(granola_ids))

## O modelo
Agora que pré-processamos nosso texto de entrada em um tensor de IDs (lembrando que cada valor de ID corresponde ao ID do token no vocabulário criado pelo tokenizador ), podemos alimentar o modelo. O modelo sabe qual palavra está sendo processada porque ele sabe qual token pertence a determinado ID. 

No BERT, assim como na maioria dos modelos de linguagem baseados em Transformers, a primeira camada é uma camada de embedding, cada token possui um embedding relacionado. No BERT, o embedding de um token é a soma de três tipos de embeddings: o embedding do token (gerado para o próprio token), o embedding do segmento (indica se o segmento faz parte da primeira ou o segunda sentença, não usado na inferência de uma única sentença) e o embedding de posição (distingue a posição do token na sentença). 

Para mais detalhes, veja [esse artigo](https://medium.com/@_init_/why-bert-has-3-embedding-layers-and-their-implementation-details-9c261108e28a). 

Abaixo, uma imagem do BERT retirada do artigo publicado.

![BERT embeddings visualization](img/bert-embeddings.png)

Para mais explicações sobre o modelo BERT, acesse [Jay Alammar's homepage](http://jalammar.github.io/).

Para começar, primeiro precisamos inicializar o modelo. Assim como o tokenizer, o modelo é pré-treinado, o que nos permite usar um modelo de linguagem já pré-treinado para obter representações de token ou de senteças.

Devemos usar o mesmo modelo pré-treinado que o tokenizer usa (`bert-base-uncased`). Este é o menor modelo BERT, treinado em texto em letras minúsculas. Desta forma, o tokenizer coloca o texto automaticamente em minúsculas para nós. A escolha do modelo, se deve ser caseado ou não, depende da tarefa. Tarefa como NER, por exemplo, podem requerer modelos treinados com maiúsculas e minúsculas.

No exemplo abaixo, um argumento adicional foi fornecido para a inicialização do modelo. `output_hidden_states` fornece mais informações de saída. Por padrão, um `BertModel` irá retornar uma tupla, mas o conteúdo dessa tupla é diferente dependendo da configuração do modelo. Ao passar `output_hidden_states = True`, a tupla irá conter:

1. O último estado oculto `(batch_size, sequence_length, hidden_size)`
2. pooler_output do token de classificação `(batch_size, hidden_size)`
3. os estados_ocultos das saídas do modelo em cada camada e as saídas dos embeddings iniciais
   `(batch_size, sequence_length, hidden_size)`

As placas gráficas (GPUs) são muito melhores em fazer operações em tensores do que uma CPU, portanto, sempre que disponível, executaremos os cálculos em GPU, como a CUDA (para isso, precisaremos de uma versão torch compatível com GPU.) 

Assim, movemos nosso modelo para o dispositivo correto: se estiver disponível, moveremos o modelo `.to ()` à GPU, caso contrário, permanecerá na CPU. É importante lembrar que o modelo e os dados a serem processados precisam estar no mesmo dispositivo. 

Finalmente, definimos o modelo para o modo de avaliação (`.eval`), em contraste com o modo de treinamento (` .train () `). Na avaliação, não temos por exemplo o dropout. 

In [5]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
# Set the device to GPU (cuda) if available, otherwise stick with CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)
granola_ids = granola_ids.to(device)

model.eval()

## Inferência

O modelo foi inicializado e a string de entrada foi convertida em um tensor. Um modelo de linguagem (como
`BertModel` acima) tem um método` forward () `que é chamado automaticamente ao chamar o objeto. O método progressivo
basicamente empurra um determinado tensor de entrada para frente no modelo e retorna a saída. Já que estamos apenas fazendo
inferir e não treinar ou ajustar o modelo, esta é a única etapa que envolve o modelo diretamente para obter
resultado. Portanto, não precisamos otimizar o modelo (calcular gradientes, propagando de volta). É muito simples, não é?
Uma peculiaridade é que definimos `torch.no_grad ()`. Isso diz ao modelo que não faremos nenhum gradiente
cálculo / retropropagação. Em última análise, torna a inferência mais rápida e mais eficiente em termos de memória. Você normalmente usaria
`model.eval ()` (veja acima) e `torch.no_grad ()` juntos para avaliação e teste de seu modelo. Ao treinar o
model deve ser definido como `model.train ()` e `torch.no_grad ()` * não * deve ser usado.

Na célula abaixo, você verá que existe um método estranho chamado `.unsqueeze ()`. Ele "descomprime" um tensor adicionando
uma dimensão extra. Em nosso caso, você verá que nosso tensor de granola de tamanho `(5,)` se transforma em uma forma diferente de
`(1, 5)` onde `1` é a dimensão da frase. Essas duas dimensões são exigidas pelo modelo: ele é otimizado
para treinar em * lotes *. O próximo parágrafo entra em um pouco mais de detalhes técnicos, mas não é necessário para entender isso
caderno.

Um lote consiste em vários textos de entrada "ao mesmo tempo" (normalmente da potência de dois, por exemplo, 64). Com um tamanho de lote
de 64 (64 frases de uma vez), o tamanho do lote seria `(64, n)` onde `64` é o número de frases e` n` o
comprimento da sequência. Neste caderno, onde usamos apenas uma entrada, o seguinte não é importante, mas se você alguma vez
deseja ajustar um modelo, você vai querer trabalhar com lotes, uma vez que o cálculo do gradiente será melhor para grandes
lotes. Nesses casos, `n` precisa ser o mesmo para todas as entradas; você não pode ter uma sequência de 5 itens e uma de
12 itens. É aí que entra o enchimento - mas isso é uma história para outro dia. Por enquanto, você pode se lembrar que o
o tamanho de entrada do modelo precisa ser `(n_input_sentences, seq_len)` onde `seq_len` pode ser determinado de diferentes maneiras.
Duas escolhas populares são: usar o texto mais longo do lote como `seq_len` (por exemplo, 12) e preencher textos mais curtos até
este comprimento, ou definindo um comprimento de sequência máximo fixo para o modelo (normalmente 512) e preencha todos os itens até este
comprimento. A última abordagem é mais fácil de implementar, mas não é eficiente em termos de memória e é computacionalmente mais pesada. o
a escolha, como sempre, é sua.


The model has been initialized, and the input string has been converted into a tensor. A language model (such as 
`BertModel` above) has a `forward()` method that is called automatically when calling the object. The forward method 
basically pushes a given input tensor forward through the model and then returns the output. Since we're only doing
inference and not training or fine-tuning the model, this is the only step that involves the model directly to get 
output. So we don't need to optimize the model (calculate gradients, propagating back). That's quite simple, isn't it?
One pecularity is that we set `torch.no_grad()`. This tells the model that we won't be doing any gradient 
calculation/backpropagation. Ultimately, it makes inference faster and more memory-efficient. You would typically use
`model.eval()` (see above) and `torch.no_grad()` together for evaluation and testing of your model. When training the
model should be set to `model.train()` and `torch.no_grad()` should *not* be used.

In the cell below, you'll see that there's a strange method called `.unsqueeze()`. It "unsqueezes" a tensor by adding 
an extra dimension. In our case, you'll see that our granola tensor of size `(5,)` turns into a different shape of
`(1, 5)` where `1` is the dimension of the sentence. These two dimensions are required by the model: it is optimised
to train on *batches*. The next paragraph goes into a bit more technical detail but is not required to understand this 
notebook.


A batch consists of multiple input texts at "the same time" (typically of the power of two, e.g. 64). With a batch size
of 64 (64 sentences at once), the batch size would be `(64, n)` where `64` is the number of sentences, and `n` the
sequence length. In this notebook, where we only ever use one input, the following is not important, but if you ever
want to fine-tune a model, you'll want to work with batches since the gradient calculation will be better for large
batches. In such cases, `n` needs to be the same for all entries; you cannot have one sequence of 5 items and one of
12 items. That is where padding comes in - but that is a story for another day. For now, you can remember that the
input size of the model needs to be `(n_input_sentences, seq_len)` where `seq_len` can be determined in different ways.
Two popular choices are: using the longest text in the batch as `seq_len` (e.g. 12) and padding shorter texts up to
this length, or setting a fixed maximal sequence length for the model (typically 512) and pad all items up to this
length. The latter approach is easier to implement but is not memory-efficient and is computationally heavier. The
choice, as always, is yours.

In [6]:
print(granola_ids.size())
# unsqueeze IDs to get batch size of 1 as added dimension
granola_ids = granola_ids.unsqueeze(0)
print(granola_ids.size())

print(type(granola_ids))
with torch.no_grad():
    out = model(input_ids=granola_ids)

# the output is a tuple
print(type(out))
# the tuple contains three elements as explained above)
print(len(out))
# we only want the hidden_states
hidden_states = out[2]
print(len(hidden_states))

As discussed above, we push the IDs of our input tokens through the `model()`, which internally calls the model's 
`forward()` method. `out` is a tuple with all relevant output items (see the list that we discussed earlier on). For us
the third item in that tuple is the most important one; it contains all `hidden_states` of the model after a forward
pass. `hidden_states` is a tuple of the output of each layer in the model for each token. In the previous
cell we saw that the tuple contains 13 items. When you execute the cell below, the architecture of the BertModel is
shown (from top-down to the bottom). The `hidden_states` include the output of the `embeddings` layer and the output of
all 12 `BertLayer`'s in the encoder. The output of each layer has a size of `(batch_size, sequence_length, 768)`.
In our case, that is `(1, 5, 768)` because we only have one input string (batch size of 1), and our input string was
tokenized into five IDs (sequence length of 5). `768` is the number of hidden dimensions.

The critical reader will notice that there is still one more layer after the encoder, called `pooler`, which is not
part of `hidden_states`. This layer is used to "pool" the output of the classification token but we will not use that 
here. Its output is returned in the second item of the output tuple `out`, as discussed before.

For an in-depth analysis of BERT's architecture, I'd 
recommend to read [the paper](https://arxiv.org/abs/1810.04805). However, if you like a more visual explanation, 
[The Illustrated BERT](http://jalammar.github.io/illustrated-bert/) might be a better place to start.

In [7]:
print(model)

Now that we have all hidden_states, we may want to get a usable value out of it. Let's say that we want to retrieve a
sentence embedding by averaging over all tokens. In other words, we want to reduce the size of `(1, 5, 768)` to
`(1, 768)` where `1` is the batch size and `768` is the number of hidden dimensions. (One could also call `768` the 
features that you wish to use in another task.) There are many ways to make a sentence abstraction of tokens, and it 
often depends on the given task. Here, we will take the mean. For now, we will only use the output of the last layer in
the encoder, that is, `hidden_states[-1]`. It is important to indicate that we want to take the `torch.mean`
_over a given axis_. Since the size of the output of the layers is `(1, 5, 768)`, we want to average over the five 
tokens, which are in the second dimension (`dim=1`). 

In [8]:
sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()
print(sentence_embedding)
print(sentence_embedding.size())

**We now have a vector of 768 features representing our input sentence.** But we can do more! The BERT paper discusses
how they reached the best results by concatenating the output of the last four layers.

![BERT embeddings visualization](img/bert-feature-extraction-contextualized-embeddings.png)

In our example, that means that
we need to get the last four layers of `hidden_states` and concatenate them after which we can take the mean. We want
to concatenate across the axis of the hidden dimensions of `768`. As a consequence, our concatenated output vector will
be of size `(1, 5, 3072)` where `3072=4*768`, i.e. the concatenation of four layers with a hidden dimension of 768. The
concatenated vector is much larger than the output of only a single layer, meaning that it contains a lot more features.
Do note, as usual, that it depends on your specific task whether these `3072` features perform better than `768`.

Having a vector of shape `(1, 5, 3072)`, we still need to take the mean over the token dimension, as we did before. We
end up with one feature vector of size `(3072,)`. 

In [9]:
# get last four layers
last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
# cast layers to a tuple and concatenate over the last dimension
cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
print(cat_hidden_states.size())

# take the mean of the concatenated vector over the token dimension
cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()
print(cat_sentence_embedding)
print(cat_sentence_embedding.size())

## Saving and loading results

It is likely that you want to use your generated feature vector in another model or task and just save them to your 
hard drive. You can easily save a tensor with `torch.save` and load it in another script with `torch.load`. Typically,
the `.pt` (PyTorch) extension is used. Note that you cannot read the saved file with a text editor. It is a pickled
object which allows for efficient (de)compression. If you do want to save your tensors in a readable format, you can
convert a tensor to numpy and using something like `np.savetxt('tensor.txt', your_tensor.numpy())`. I do not recommend
that approach (I'd stick with `torch.save` or another compression technique) but it is possible.

See how we use `.cpu()`? `cpu()` tells PyTorch that we want to move the output tensor back from the GPU to the CPU. 
This is not a required step, but I think it is good practice when doing feature extraction to move your data to CPU so
that when you load it, it is also loaded as a CPU tensor rather than a CUDA tensor. Afterwards you can still move 
things to GPU if need be, but using CPU by default seems like a good idea. Note that a tensor has to be on CPU if you
want to convert it to `.numpy()`, though.

In [10]:
# save our created sentence representation
torch.save(cat_sentence_embedding.cpu(), 'my_sent_embed.pt')

# load it again
loaded_tensor = torch.load('my_sent_embed.pt')
print(loaded_tensor)
print(loaded_tensor.size())

# convert it to numpy to use in e.g. sklearn
np_loaded_tensor = loaded_tensor.numpy()
print(np_loaded_tensor)
print(type(np_loaded_tensor))
